# Spark GraphFrame


## Import modules

In [1]:
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql.functions import col, lit, when

## Session

In [2]:
spark = SparkSession.builder \
                    .master("local[4]") \
                    .appName("spark graphframe tutorial") \
                    .config("spark.executor.memory", "1g") \
                    .config("spark.jars.packages", "graphframes:graphframes:0.7.0-spark2.4-s_2.11") \
                    .getOrCreate()

In [3]:
from graphframes import *

In [4]:
spark.conf.set("spark.sql.shuffle.partitions", "4")

## Context

In [5]:
sc = spark.sparkContext

## Creating GraphFrames

In [6]:
vertices = spark.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
  ("d", "David", 29),
  ("e", "Esther", 32),
  ("f", "Fanny", 36),
  ("g", "Gabby", 60)], ["id", "name", "age"])

In [7]:
edges = spark.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
  ("f", "c", "follow"),
  ("e", "f", "follow"),
  ("e", "d", "friend"),
  ("d", "a", "friend"),
  ("a", "e", "friend")
], ["src", "dst", "relationship"])

In [13]:
g = GraphFrame(vertices, edges)
print g

GraphFrame(v:[id: string, name: string ... 1 more field], e:[src: string, dst: string ... 1 more field])


## Basic graph and DataFrame queries

In [11]:
g.vertices.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 30|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 36|
|  g|  Gabby| 60|
+---+-------+---+



In [12]:
g.edges.show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  c|      follow|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
|  e|  d|      friend|
|  d|  a|      friend|
|  a|  e|      friend|
+---+---+------------+



In [14]:
g.inDegrees.show()

+---+--------+
| id|inDegree|
+---+--------+
|  f|       1|
|  b|       2|
|  c|       2|
|  d|       1|
|  a|       1|
|  e|       1|
+---+--------+



In [15]:
g.outDegrees.show()

+---+---------+
| id|outDegree|
+---+---------+
|  f|        1|
|  b|        1|
|  a|        2|
|  c|        1|
|  e|        2|
|  d|        1|
+---+---------+



In [16]:
g.degrees.show()

+---+------+
| id|degree|
+---+------+
|  f|     2|
|  b|     3|
|  a|     3|
|  c|     3|
|  e|     3|
|  d|     2|
+---+------+



In [17]:
g.vertices.groupBy().min("age").show()

+--------+
|min(age)|
+--------+
|      29|
+--------+



In [18]:
numFollows = g.edges.filter("relationship = 'follow'").count()
print("The number of follow edges is", numFollows)

('The number of follow edges is', 4)


## Motif finding
- Motif finding refers to searching for structural patterns in a graph.

In [19]:
# search for pairs of vertices a, b connected by edges in both directions
motifs = g.find("(a)-[e]->(b); (b)-[e2]->(a)")
motifs.show()

+----------------+--------------+----------------+--------------+
|               a|             e|               b|            e2|
+----------------+--------------+----------------+--------------+
|[c, Charlie, 30]|[c, b, follow]|    [b, Bob, 36]|[b, c, follow]|
|    [b, Bob, 36]|[b, c, follow]|[c, Charlie, 30]|[c, b, follow]|
+----------------+--------------+----------------+--------------+



In [22]:
motifs.filter("b.age > 30 or a.age > 30").show()

+----------------+--------------+----------------+--------------+
|               a|             e|               b|            e2|
+----------------+--------------+----------------+--------------+
|[c, Charlie, 30]|[c, b, follow]|    [b, Bob, 36]|[b, c, follow]|
|    [b, Bob, 36]|[b, c, follow]|[c, Charlie, 30]|[c, b, follow]|
+----------------+--------------+----------------+--------------+



In [23]:
motifs.filter("b.age > 30").show()

+----------------+--------------+------------+--------------+
|               a|             e|           b|            e2|
+----------------+--------------+------------+--------------+
|[c, Charlie, 30]|[c, b, follow]|[b, Bob, 36]|[b, c, follow]|
+----------------+--------------+------------+--------------+



## Subgraphs

In [24]:
g2 = g.filterEdges("relationship = 'friend'").filterVertices("age > 30").dropIsolatedVertices()

In [25]:
g2.vertices.toPandas()

,id,name,age
0,b,Bob,36
1,a,Alice,34
2,e,Esther,32


In [26]:
g2.edges.toPandas()

,src,dst,relationship
0,a,b,friend
1,a,e,friend


## Graph algorithms
- GraphFrames comes with a number of standard graph algorithms built in:
  - Breadth-first search (BFS)
  - Connected components
  - Strongly connected components
  - Label Propagation Algorithm (LPA)
  - PageRank (regular and personalized)
  - Shortest paths
  - Triangle count

### Breadth-first search (BFS)

In [27]:
paths = g.bfs("name = 'Esther'", "age < 32")
paths.toPandas()

,from,e0,to
0,"(e, Esther, 32)","(e, d, friend)","(d, David, 29)"


### PageRank
Identify important vertices in a graph based on connections.

In [28]:
results = g.pageRank(resetProbability=0.15, tol=0.01)

In [29]:
results.vertices.toPandas()

,id,name,age,pagerank
0,a,Alice,34,0.449106
1,e,Esther,32,0.370852
2,f,Fanny,36,0.328361
3,d,David,29,0.328361
4,c,Charlie,30,2.687830
5,b,Bob,36,2.655508
6,g,Gabby,60,0.179982


In [30]:
results.edges.toPandas()

,src,dst,relationship,weight
0,b,c,follow,1.0
1,c,b,follow,1.0
2,f,c,follow,1.0
3,a,b,friend,0.5
4,d,a,friend,1.0
5,e,f,follow,0.5
6,a,e,friend,0.5
7,e,d,friend,0.5


In [32]:
# Run PageRank for a fixed number of iterations.
results = g.pageRank(resetProbability=0.15, maxIter=10)

In [33]:
results.vertices.toPandas()

,id,name,age,pagerank
0,a,Alice,34,0.448512
1,e,Esther,32,0.361349
2,f,Fanny,36,0.325049
3,d,David,29,0.325049
4,c,Charlie,30,2.666788
5,b,Bob,36,2.702522
6,g,Gabby,60,0.170732


In [34]:
results.edges.toPandas()

,src,dst,relationship,weight
0,b,c,follow,1.0
1,c,b,follow,1.0
2,f,c,follow,1.0
3,a,b,friend,0.5
4,d,a,friend,1.0
5,e,f,follow,0.5
6,a,e,friend,0.5
7,e,d,friend,0.5


In [35]:
# Run PageRank personalized for vertex "a"
results = g.pageRank(resetProbability=0.15, maxIter=10, sourceId="a")

In [36]:
results.vertices.toPandas()

,id,name,age,pagerank
0,a,Alice,34,0.177108
1,e,Esther,32,0.076578
2,f,Fanny,36,0.031892
3,d,David,29,0.031892
4,c,Charlie,30,0.345915
5,b,Bob,36,0.336614
6,g,Gabby,60,0.000000


In [37]:
results.edges.toPandas()

,src,dst,relationship,weight
0,b,c,follow,1.0
1,c,b,follow,1.0
2,f,c,follow,1.0
3,a,b,friend,0.5
4,d,a,friend,1.0
5,e,f,follow,0.5
6,a,e,friend,0.5
7,e,d,friend,0.5


### Shortest paths
Computes shortest paths from each vertex to the given set of landmark vertices, where landmarks are specified by vertex ID.

In [38]:
results = g.shortestPaths(landmarks=["a", "d"])
results.toPandas()

,id,name,age,distances
0,a,Alice,34,"{u'a': 0, u'd': 2}"
1,e,Esther,32,"{u'a': 2, u'd': 1}"
2,f,Fanny,36,{}
3,d,David,29,"{u'a': 1, u'd': 0}"
4,c,Charlie,30,{}
5,b,Bob,36,{}
6,g,Gabby,60,{}


### Triangle count
Computes the number of triangles passing through each vertex.

In [39]:
results = g.triangleCount()
results.toPandas()

,count,id,name,age
0,0,f,Fanny,36
1,0,b,Bob,36
2,0,g,Gabby,60
3,1,a,Alice,34
4,0,c,Charlie,30
5,1,d,David,29
6,1,e,Esther,32


## Reference

- https://graphframes.github.io/graphframes/docs/_site/index.html
- https://docs.databricks.com/spark/latest/graph-analysis/graphframes/user-guide-python.html